In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

from math import sqrt
from sklearn.metrics import mean_squared_error

%matplotlib inline
import matplotlib.pyplot as plt  
import seaborn as sns
import statsmodels.api as sm

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# **Input data**

In [ ]:
sample = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/sample_submission.csv')
train = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv',parse_dates=['date'])
test = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv',parse_dates=['date'])

# **Explore data**

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train.skew()

Skewnwss is between -1 to 1, consider as no outliers.

In [ ]:
test.describe()

In [ ]:
test[88:92]

In [ ]:
print("Min date from train set: {}".format(train.date.min()))
print("Max date from train set: {}".format(train.date.max()))
print("Min date from test set: {}".format(test.date.min()))
print("Max date from test set: {}".format(test.date.max()))

In [ ]:
sample.head()

# **Visualize data: Daily Sales by Store**
* From data explore: there're 10 stores (1-10) and no missing values

In [ ]:
store_sales = train.groupby(['date','store'], as_index = False)['sales'].sum()
store_sales_plot = []
for i in range(1,11):
    current_store = store_sales[(store_sales['store'] == i)]
    store_sales_plot.append(go.Scatter(x = current_store['date'], y = current_store['sales'],
                                       name = ('store %s' % i)))
fig = go.Figure(data = store_sales_plot)
iplot(fig)

From the plot we can see the 10 stores have similar trend and seasonality, higher sales in middle of the year and end of the year, and there's increase in sales year by year.

# **Visualize data: Daily Sales by Item**

* From data explore: there're 50 items (1-50) and no missing values

In [ ]:
item_sales = train.groupby(['date','item'], as_index = False)['sales'].sum()
item_sales_plot = []
for i in range(1,51):
    current_item = item_sales[item_sales['item'] == i]
    item_sales_plot.append(go.Scatter(x = current_item['date'], y = current_item['sales'],
                                   name = 'item %s' % i))
fig = go.Figure(data = item_sales_plot)
iplot(fig)

From the plot we can see the 50 items have similar trend and seasonality, higher sales in middle of the year and end of the year, and there's increase in sales year by year.

# **Build model using SARIMA**

In [ ]:
!pip install pmdarima
from pmdarima import auto_arima

Random choose a store:

In [ ]:
np.random.randint(low=1, high=10, size=(1,))

Random choose an item:

In [ ]:
np.random.randint(low=1, high=50, size=(1,))

Run auto arima for store 5 and item 29 (use last 90 days sales as validation):

In [ ]:
train_1 = train[(train['store'] == 5) & (train['item'] == 29)]

train_1_df = train_1[:(len(train_1)-90)]
valid_1_df = train_1[len(train_1)-90:len(train_1)]

train_1_df.drop(columns = ['store','item'], inplace=True)
valid_1_df.drop(columns = ['store','item'], inplace=True)

train_1_df = train_1_df.set_index('date')
valid_1_df = valid_1_df.set_index('date')

autoarima_model = auto_arima(train_1_df,m=7,
                             seasonal=True,
                             trace=True,
                             error_action='ignore',
                             suppress_warnings=True,
                             stepwise=True)
autoarima_model.fit(train_1_df)
forecast = autoarima_model.predict(n_periods=len(valid_1_df))
rmse = sqrt(mean_squared_error(valid_1_df,forecast))

In [ ]:
rmse

From auto arima, the best fit model is (2,1,1)x(2,0,2,7) and rmse is 12.86.
As each store and item has similar trend and seasonality, can use the same SARIMA model for all stores and items.
Maybe it will be better to use auto arima for each store and item, but it will take very long.

In [ ]:
train_df = train.set_index('date')
rmse_list = []
start_index = '2017-10-01'
end_index = '2017-12-31'
actualvsforecast = pd.DataFrame()

for i in range(1,51):
    for j in range(1,11):
        train_1 = train_df[(train_df['store'] == j) & (train_df['item'] == i)]
        model = sm.tsa.statespace.SARIMAX(train_1.sales[:start_index],
                                          order=(2,1,1),
                                          seasonal_order=(2,0,2,7),
                                          freq='D').fit()
        train_1['forecast'] = model.predict(start = pd.to_datetime(start_index), 
                                            end = pd.to_datetime(end_index),
                                            dynamic = True)
        rmse = sqrt(mean_squared_error(train_1[start_index:end_index]['sales'],
                                       train_1[start_index:end_index]['forecast']))
        rmse_list.append(rmse)
        actualvsforecast = actualvsforecast.append(train_1[start_index:end_index])

In [ ]:
actualvsforecast.to_csv("actualvsforecast_arima.csv")

In [ ]:
train_1['forecast'].count()/len(train_1)

This 0.05 percentage will be used to split train and test for XGBoost model.

In [ ]:
len(rmse_list)

In [ ]:
sum(rmse_list)/len(rmse_list)

The average rmse is 13.56.

# **Build model using XGBoost**

In [ ]:
train['date']=pd.to_datetime(train['date'])
train['weekday']=train['date'].dt.dayofweek
train['dayofyear']=train['date'].dt.dayofyear
train['year']=train['date'].dt.year
train['month']=train['date'].dt.month

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
X=train.copy()
X.drop(['sales','date'],axis=1,inplace=True)
y=train['sales']

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.05,random_state=123)

xgb=XGBRegressor(random_state=123)


XBG_score=cross_val_score(xgb,X_train,y_train,scoring='neg_mean_squared_error')

In [ ]:
print("RMSE:",np.sqrt(-XBG_score.mean()))

rmse is 7.73, which is lower than SARIMA mode rmse (13.56). Hence choose XGBoost model to predict.

# **Prediction using test file**

In [ ]:
xgb.fit(X,y)

In [ ]:
test['weekday']=test['date'].dt.dayofweek
test['dayofyear']=test['date'].dt.dayofyear
test['year']=test['date'].dt.year
test['month']=test['date'].dt.month

X=test.copy()
X.drop(['id','date'],axis=1,inplace=True)

test_predictions=xgb.predict(X)

In [ ]:
final = pd.DataFrame()
final['id']=test['id']
final['sales']=test_predictions

In [ ]:
final.head()

In [ ]:
final.to_csv("submission.csv", index=False)